In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import KFold
import keras
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.callbacks import EarlyStopping



In [ ]:
# Link προς το csv αρχείο
url = 'https://raw.githubusercontent.com/vgkourgkoutas/Computational_Intelligence/main/iphi2802.csv'
# Διαβάζουμε το αρχείο CSV με τις επιγραφές και τις χρονολογίες
df = pd.read_csv(url, delimiter='\t')
df.head()

In [ ]:
texts = df["text"]

# Ορίζουμε τον TfidfVectorizer με τις κατάλληλες παραμέτρους
# Δημιοργία του μοντέλου BoW (Bag of Words) με unigrams με κωδικοποίηση TF-IDF και επιλογή μόνο των 1000 σημαντικότερων λέξεων
vectorizer = TfidfVectorizer(analyzer='word',max_features=1000)

# Εφαρμόζουμε τον vectorizer στις επιγραφές για να πάρουμε το BoW representation
X = vectorizer.fit_transform(texts)

# Εκτυπώνουμε το λεξικό που έχουμε δημιουργήσει
print(vectorizer.vocabulary_)

print()

#Εκτυπώνουμε το idf για κάθε λέξη:
all_feature_names = vectorizer.get_feature_names_out()

for word in all_feature_names:


    indx = vectorizer.vocabulary_.get(word)

    #Λαμβάνουμε το score για κάθε λέξη
    idf_score = vectorizer.idf_[indx]

    print(f"{word} : {idf_score}")



# Δημιουργία ενός αντικειμένου MinMaxScaler
scaler = MinMaxScaler()

X_dense = X.toarray()

# Κανονικοποίηση των δεδομένων στο εύρος [0,1]
X_normalized = scaler.fit_transform(X_dense)

print()
print()
print("Normalized data for the input X:")
print(X_normalized.shape)
print(X_normalized)

# Προσθήκη μιας νέας στήλης "average_date" που περιέχει τον μέσο όρο των ημερομηνιών
y = df['average_date'] = (df['date_min'] + df['date_max']) / 2

# Δημιουργία του scaler
scaler = MinMaxScaler()

# Κανονικοποίηση του y στο εύρος [0, 1]
y_normalized = scaler.fit_transform(y.values.reshape(-1, 1))

print()
print("Normalized data for the output y:")
print(y_normalized.shape)
print(y_normalized)


In [ ]:
# Εκτύπωση του συνολικού μεγέθους του dataset
print("Συνολικό μέγεθος του dataset:", df.shape[0])

# Δημιουργία του KFold splitter με 5 folds
kf = KFold(n_splits=5, shuffle=False)

# Επιβεβαίωση διαχωρισμού δεδομένων με 5 cross-validation
fold = 1
for train_index, test_index in kf.split(X_normalized):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y_normalized[train_index], y_normalized[test_index]

    print(f"Fold {fold}:")
    print("Train set size for X:", X_train.shape[0])
    print("Test set size for X:", X_test.shape[0])
    print("Train set size for y:", y_train.shape[0])
    print("Test set size for y:", y_test.shape[0])


    print()

    fold += 1


In [ ]:
# Ερώτημα Α2 με διαφορετικό αριθμό νευρώνων

# Συνάρτηση που υλοποιεί το Μέσο τετραγωνικό σφάλμα (RMSE)
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Λίστα για αποθήκευση του RMSE για κάθε τιμή νευρώνων
rmse_results = {}

# Διαφορετικές τιμές νευρώνων που θα δοκιμάσουμε
hidden_layer_neurons = [20, 80, 170]

# Λίστα για αποθήκευση της απώλειας για κάθε fold σε κάθε τιμή νευρώνων
convergence_results = {}

# Δοκιμή για κάθε τιμή νευρώνων
for neurons in hidden_layer_neurons:
    print(f"\nTesting with {neurons} neurons in hidden layer:")
    rmseList = []
    convergence = []

    for i, (train_index, test_index) in enumerate(kf.split(X_normalized)):
        X_train, X_test = X_normalized[train_index], X_normalized[test_index]
        y_train, y_test = y_normalized[train_index], y_normalized[test_index]

        # Δημιουργία μοντέλου
        model = Sequential()
        model.add(Dense(neurons, activation="relu"))
        model.add(Dense(1, activation="linear"))

        # Compile του μοντέλου
        adam_optimizer = keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='mean_squared_error', optimizer=adam_optimizer, metrics=[rmse])

        # Προσαρμογή του μοντέλου και αποθήκευση του rmse για κάθε εποχή
        history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), verbose=0)
        convergence.append(history.history["rmse"])

        # Αξιολόγηση του μοντέλου
        scores_test = model.evaluate(X_test, y_test, verbose=0)
        rmseList.append(scores_test[1])

        print(f"Fold {i+1}: RMSE: {scores_test[1]}")

    # Αποθήκευση του μέσου RMSE και της σύγκλισης
    rmse_results[neurons] = {
        "average_rmse": np.mean(rmseList),
        "convergence": np.mean(np.array(convergence), axis=0)  # Μέσος όρος των απωλειών ανά epoch
    }


    # Αποθήκευση της σύγκλισης για κάθε fold
    convergence_results[neurons] = convergence

# Εκτύπωση των αποτελεσμάτων για κάθε τιμή νευρώνων
for neurons, result in rmse_results.items():
    print(f"\nAverage RMSE with {neurons} neurons: {result['average_rmse']}")

# Δημιουργία γραφικών παραστάσεων σύγκλισης για κάθε τιμή νευρώνων
plt.figure(figsize=(10, 6))
for neurons, result in rmse_results.items():
    plt.plot(result["convergence"], label=f"{neurons} Neurons")
plt.xlabel("Epoch")
plt.ylabel("Average Rmse")
plt.title("Convergence of Training for Different Hidden Layer Neurons")
plt.legend()
plt.show()

# Δημιουργία γραφικών παραστάσεων για κάθε διαφορετικό αριθμό νευρώνων
for neurons in hidden_layer_neurons:
    plt.figure(figsize=(10, 6))
    plt.title(f"Convergence for {neurons} Hidden Layer Neurons")
    plt.xlabel("Epoch")
    plt.ylabel("RMSE")

    # Προσθήκη των τιμών απώλειας για κάθε fold στη γραφική παράσταση
    for fold, rmse in enumerate(convergence_results[neurons]):
        plt.plot(rmse, label=f"Fold {fold + 1}")

    plt.legend()
    plt.show()


In [ ]:
# Προσθήκη περισσότερων κρυφών επιπέδων για το ερώτημα Α2,ε

from keras.callbacks import EarlyStopping

# Συνάρτηση που υλοποιεί το Μέσο τετραγωνικό σφάλμα (RMSE)
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Λίστα για αποθήκευση του RMSE για κάθε τιμή νευρώνων
rmse_results = {}

# Λίστα για αποθήκευση της απώλειας για κάθε fold σε κάθε τιμή νευρώνων
convergence_results = {}

rmseList = []
convergence = []

for i, (train_index, test_index) in enumerate(kf.split(X_normalized)):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y_normalized[train_index], y_normalized[test_index]

    # Create model
    model = Sequential()
    model.add(Dense(500, activation="relu"))
    model.add(Dense(800, activation="relu"))
    model.add(Dense(1000, activation="relu"))
    model.add(Dense(1, activation="linear"))

    # Compile the model
    adam_optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=adam_optimizer, metrics=[rmse])

    # Fit model and store the loss for each epoch
    history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),verbose=0)
    convergence.append(history.history["val_rmse"])

    # Evaluate model
    scores_test = model.evaluate(X_test, y_test, verbose=0)
    rmseList.append(scores_test[1])

    print(f"Fold {i+1}: RMSE: {scores_test[1]}")

print(f"\nAverage RMSE:", np.mean(rmseList))


In [ ]:
# Προσθήκη early stopping για το Α2,στ

# Συνάρτηση που υλοποιεί το Μέσο τετραγωνικό σφάλμα (RMSE)
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Callback για πρόωρο σταμάτημα
early_stopping = EarlyStopping(
    monitor='val_rmse',  # Παρακολουθούμε το RMSE στο validation set
    patience=10,  # Σταματάμε αν δεν υπάρξει βελτίωση μετά από ένα συγκεκριμένο όριο εποχών
    #min_delta=0.01,  # Η ελάχιστη βελτίωση για να θεωρηθεί αλλαγή
    mode='min',  # Αναζητούμε το ελάχιστο RMSE
    verbose=1  # Εμφανίζουμε μήνυμα όταν σταματά
)

# Λίστα για αποθήκευση του RMSE και των τιμών απώλειας για κάθε fold
rmseList = []
convergence = []

for i, (train_index, test_index) in enumerate(kf.split(X_normalized)):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y_normalized[train_index], y_normalized[test_index]

    # Δημιουργία του μοντέλου
    model = Sequential()
    model.add(Dense(500, activation="relu"))
    model.add(Dense(800, activation="relu"))
    model.add(Dense(1000, activation="relu"))
    model.add(Dense(1, activation="linear"))

    # Compile του μοντέλου
    adam_optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='mean_squared_error',
        optimizer=adam_optimizer,
        metrics=[rmse]
    )

    # Εκπαίδευση με πρόωρο σταμάτημα
    history = model.fit(X_train,y_train,epochs=20,validation_data=(X_test, y_test),callbacks=[early_stopping],verbose=0)

    convergence.append(history.history["rmse"])

    # Αξιολόγηση του μοντέλου στο test set
    scores_test = model.evaluate(X_test, y_test, verbose=0)
    rmseList.append(scores_test[1])

    print(f"Fold {i+1}: RMSE: {scores_test[1]}")

print(f"\nAverage RMSE:", np.mean(rmseList))


# Δημιουργία γραφικών παραστάσεων για κάθε διαφορετικό αριθμό νευρώνων
plt.figure(figsize=(10, 6))
plt.title(f"Convergence for 3 Hidden Layers")
plt.xlabel("Epoch")
plt.ylabel("RMSE")

# Προσθήκη των τιμών απώλειας για κάθε fold στη γραφική παράσταση
for fold, loss in enumerate(convergence):
    plt.plot(loss, label=f"Fold {fold + 1}")

plt.legend()
plt.show()

In [ ]:
# Ερώτημα Α3
# Αλλάζουμε χειροκίνητα τα ορίσματα learning_rate και beta_l (momentum) και επιλέγουμε τον καλύτερο συνδυασμό από τον πίνακα (learning_rate=0.001 και momentum = 0.6)

# Συνάρτηση που υλοποιεί το Μέσο τετραγωνικό σφάλμα (RMSE)
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Callback για πρόωρο σταμάτημα
early_stopping = EarlyStopping(
    monitor='val_rmse',  # Παρακολουθούμε το RMSE στο validation set
    patience=10,  # Σταματάμε αν δεν υπάρξει βελτίωση μετά από ένα συγκεκριμένο όριο εποχών
    #min_delta=0.01,  # Η ελάχιστη βελτίωση για να θεωρηθεί αλλαγή
    mode='min',  # Αναζητούμε το ελάχιστο RMSE
    verbose=1  # Εμφανίζουμε μήνυμα όταν σταματά
)

# Λίστα για αποθήκευση του RMSE και των τιμών απώλειας για κάθε fold
rmseList = []
convergence = []

for i, (train_index, test_index) in enumerate(kf.split(X_normalized)):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y_normalized[train_index], y_normalized[test_index]

    # Δημιουργία του μοντέλου
    model = Sequential()
    model.add(Dense(500, activation="relu"))
    model.add(Dense(800, activation="relu"))
    model.add(Dense(1000, activation="relu"))
    model.add(Dense(1, activation="linear"))

    # Compile του μοντέλου με τον optimizer Adam
    adam_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.6)
    model.compile(
        loss='mean_squared_error',
        optimizer=adam_optimizer,
        metrics=[rmse]
    )

    # Εκπαίδευση με πρόωρο σταμάτημα
    history = model.fit(X_train,y_train,epochs=20,validation_data=(X_test, y_test),callbacks=[early_stopping],verbose=0)

    convergence.append(history.history["rmse"])

    # Αξιολόγηση του μοντέλου στο test set
    scores_test = model.evaluate(X_test, y_test, verbose=0)
    rmseList.append(scores_test[1])

    print(f"Fold {i+1}: RMSE: {scores_test[1]}")

print(f"\nAverage RMSE:", np.mean(rmseList))


# Δημιουργία γραφικών παραστάσεων για κάθε διαφορετικό αριθμό νευρώνων
plt.figure(figsize=(10, 6))
plt.title(f"Convergence for 3 Hidden Layers")
plt.xlabel("Epoch")
plt.ylabel("RMSE")

# Προσθήκη των τιμών απώλειας για κάθε fold στη γραφική παράσταση
for fold, rmse in enumerate(convergence):
    plt.plot(rmse, label=f"Fold {fold + 1}")

plt.legend()
plt.show()

In [ ]:
#Εκτέλεση του μοντέλου για όλους του συνδυασμούς του ερωτήματος Α4

# Συνάρτηση RMSE
def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

# Καθορισμός των τιμών του rin και του rh
rin_values = [0.8, 0.5, 0.8]
rh_values = [0.5, 0.5, 0.2]

# Δημιουργία λίστας για αποθήκευση των αποτελεσμάτων
results = []

# Δοκιμή κάθε συνδυασμού των πιθανών συντελεστών
for rin, rh in zip(rin_values, rh_values):
    rmse_list = []
    convergence = []

    # Early stopping με patience 10
    early_stopping = EarlyStopping(monitor='val_rmse', patience=10, verbose=1, mode='min')

    for train_index, test_index in kf.split(X_normalized):
        X_train, X_test = X_normalized[train_index], X_normalized[test_index]
        y_train, y_test = y_normalized[train_index], y_normalized[test_index]

        # Δημιουργία του μοντέλου με Dropout
        model = Sequential()
        model.add(Dropout(1 - rin, input_shape=(X_train.shape[1],)))  # Dropout για τους κόμβους εισόδου
        model.add(Dense(500, activation='relu'))
        model.add(Dropout(1 - rh))  # Dropout για τους κρυφούς κόμβους
        model.add(Dense(800, activation='relu'))
        model.add(Dropout(1 - rh))
        model.add(Dense(1000, activation='relu'))
        model.add(Dense(1, activation='linear'))

        # Compile του μοντέλου
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.6)
        model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[rmse])

        # Εκπαίδευση του μοντέλου με early stopping
        history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)

        convergence.append(history.history["rmse"])

        # Αξιολόγηση στο test set
        scores_test = model.evaluate(X_test, y_test, verbose=0)
        rmse_list.append(scores_test[1])

    # Αποθήκευση των αποτελεσμάτων για κάθε συνδυασμό
    avg_rmse = np.mean(rmse_list)
    results.append((rin, rh, avg_rmse, convergence))

# Δημιουργία γραφικών παραστάσεων για κάθε συνδυασμό που δείχνουν τα folds
for (rin, rh, avg_rmse, convergence) in results:
    plt.figure(figsize=(10, 6))
    plt.title(f"Convergence for rin: {rin}, rh: {rh}")
    plt.xlabel("Epoch")
    plt.ylabel("Validation RMSE")

    # Προσθήκη όλων των γραφικών παραστάσεων που δείχνουν τα folds
    for fold, convergence_fold in enumerate(convergence):
        plt.plot(convergence_fold, label=f"Fold {fold + 1}")

    plt.legend()
    plt.show()

# Εκτύπωση των αποτελεσμάτων για κάθε συνδυασμό
results_summary = "\n".join([f"rin: {rin}, rh: {rh}, Average RMSE: {avg_rmse:.4f}" for rin, rh, avg_rmse, _ in results])
print("Average RMSE for each combination:")
print(results_summary)


In [ ]:
#Επιλογή του καλύτερου συνδυασμού έπειτα από την εκτέλεση του μοντέλου στο ερώτημα Α4 (επιλογή του rin=0.5 και rh=0.5)

# Συνάρτηση RMSE
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Καθορισμός των διαφόρων συντελεστών διατήρησης για το dropout
rin = 0.5 # Συντελεστής για τους κόμβους εισόδου
rh = 0.5 # Συντελεστής για τους κρυφούς κόμβους


rmseList = []
convergence = []
# Early stopping με patience 10
early_stopping = EarlyStopping(monitor='val_rmse', patience=10, verbose=1, mode='min')

for i, (train_index, test_index) in enumerate(kf.split(X_normalized)):
    X_train, X_test = X_normalized[train_index], X_normalized[test_index]
    y_train, y_test = y_normalized[train_index], y_normalized[test_index]

    # Δημιουργία του μοντέλου με Dropout
    model = Sequential()
    model.add(Dropout(1 - rin, input_shape=(X_train.shape[1],)))  # Dropout για τους κόμβους εισόδου
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(1 - rh))  # Dropout για τους κρυφούς κόμβους
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(1 - rh))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(1, activation='linear'))

    # Compile του μοντέλου
    optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.6)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[rmse])

    # Εκπαίδευση με early stopping
    history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=0)

    convergence.append(history.history["rmse"])

    # Αξιολόγηση στο test set
    scores_test = model.evaluate(X_test, y_test, verbose=0)
    rmseList.append(scores_test[1])

    print(f"Fold {i+1}: RMSE: {scores_test[1]}")


# Αποθήκευση των αποτελεσμάτων για τον συνδυασμό rin, rh
avg_rmse = np.mean(rmseList)

print(f"\nAverage RMSE:", avg_rmse)


# Δημιουργία γραφικών παραστάσεων για κάθε διαφορετικό αριθμό νευρώνων
plt.figure(figsize=(10, 6))
plt.title(f"Convergence for 3 Hidden Layers")
plt.xlabel("Epoch")
plt.ylabel("RMSE")

# Προσθήκη των τιμών απώλειας για κάθε fold στη γραφική παράσταση
for fold, rmse in enumerate(convergence):
    plt.plot(rmse, label=f"Fold {fold + 1}")

plt.legend()
plt.show()
